# python排序

## [].sort排序

默认的自然排序，效率还不错

## 自定义函数排序

列表排序的顺序只是适用自然顺序，很多时候，你需要特定的顺序。比如需要排序的字段不是第一个字符。

`cmp()`就是`[].sort()`的默认比较函数（**在速度上'list.sort()'远远超过'list.sort(cmp)'**）。对于不太长的列表使用自定义比较函数可以快速的解决问题。在很多情况下，甚至可以直接使用一个'lambda'表达式来完成任务。

说到速度，使用自定义比较函数效率会很低。部分原因是**Python的函数调用开销，函数本身也会增加花费的时间**。不过有一种技术“Schwartzian转换”可以加速这种自定义排序。

# Schwartzian变换原理

使用Schwartzian转换主要包括三个步骤，（准确的来说这是Guttman-Rosler转换(GRT)，同样基于Schwartzian转换）

1.将列表转换为可以用默认排序的列表

2.使用`[].sort()`排序

3.转回原先的格式

核心是**将需要排序的对象转为一个能进行自然排序的字符串**，任务里排序时间是主要因素的话，使用这项技术将大大提高效率（唯一的限制就是转换花费的时间不会很多）

In [ ]:
import sys,string,time,datatime
wrerr=sys.stderr.write
def get_Time(str):
    print(len(str))
    print(str)
    time=datetime.datetime.strprime(str,'%d%b%Y:%H:%M:%S')
    return time.strftime("%Y%m%d%H%M%S")

lines=open(sys.argv[1]).readlines()
start=time.time()
for n in range(len(lines)):
    lst=string.split(lines[n])
    lines[n]=(get_Time(lines[n][:20]),lines[n])
lines.sort()
for n in range(len(lines)):
    lines[n]=lines[n][1]
end=time.time()
wrerr("Schwartzian transform sort in %3.2f secs\n" %(end-start))
open('time.schwartzian','w').writelines(lines)

核心在于get_Time函数，讲时间转换为可以自然排序的字符串。

# python排序

使用列表本身的方法**list.sort()**去排序。它会改变list的内容，然后返回None作为执行的结果，以避免混淆。一般来说它没有sorted()那么方便，但是如果你不需要原来的列表的话，使用它在**性能上会有轻微的提升**。

**list.sort()方法只能用于列表，相对的，sorted()函数则适用于所有的可迭代对象**

In [6]:
sorted({1:'D',2:'B',3:'B',4:'E',5:'A'})

[1, 2, 3, 4, 5]

key参数对应的值，必须是这样一个函数：接受一个参数然后返回一个用来排序的键。用这种技术来排序在速度上是非常快的，因为**key函数恰好被每一个输入记录调用一次。**

In [1]:
student_tuples=[('john','A',15),('jane','B',12),('dave','B',10)]

In [2]:
sorted(student_tuples,key=lambda student:student[2])

[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

In [3]:
class Student:
    def __init__(self,name,grade,age):
        self.name=name
        self.grade=grade
        self.age=age
    def __repr__(self):
        return repr((self.name,self.grade,self.age))

In [4]:
student_objects=[Student('john','A',15),Student('jane','B',12),Student('dave','B',10)]

In [5]:
sorted(student_objects,key=lambda student:student.age)

[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]

key函数不一定要直接基于被排序的对象。**key函数同样可以访问外部的资源**。

In [17]:
students=['dave','john','jane']
grades={'john':'F','jane':'A','dave':'C'}
sorted(students,key=grades.__getitem__)

['jane', 'dave', 'john']

Python本身也提供了很多很方便的函数，让创建访问器函数变得更快、更容易。**operater模块提供的函数有operator.itemgetter()，operator.attrgetter()，operator.methodcaller()函数。**

operator模块提供的几个函数**允许多级别的排序**

In [10]:
from operator import itemgetter,attrgetter
sorted(student_tuples, key=itemgetter(1,2))

[('john', 'A', 15), ('dave', 'B', 10), ('jane', 'B', 12)]

In [11]:
sorted(student_objects, key=attrgetter('grade','age'))

[('john', 'A', 15), ('dave', 'B', 10), ('jane', 'B', 12)]

operator.methodcaller()函数会**在每个被排序的对象上执行一个固定参数的方法调用**

In [13]:
from operator import methodcaller
messages=['critical!!!', 'hurry!', 'standby', 'immediate!!']
sorted(messages, key=methodcaller('count', '!'))

['standby', 'hurry!', 'immediate!!', 'critical!!!']

**排序都是稳定**

**Python的Timsort算法可以高效率地进行多重排序，因为它能很好的利用数据集中已经存在的有序序列。**

# 使用decorate-sort-undecorated方法

这个习语是以它的三个步骤而被命名为decorate-sort-undecorate（装饰-排序-去装饰）的

1.原始的列表被装饰以生成新的值，这些值是用来控制排序顺序的。

2.对被装饰过的列表进行排序。

3.去掉装饰，以新的顺序创建一个列表，这个列表只包含原来列表中的值。

In [15]:
decorated = [(student.grade, i, student) for i, student in enumerate(student_objects)]
decorated.sort()
[student for grade, i, student in decorated] 

[('john', 'A', 15), ('jane', 'B', 12), ('dave', 'B', 10)]

在这里，元组是按照字典序进行比较的，首先第一项被比较，如果他们相等则对第二项进行比较，以此类推。

不是在所有的情况下都需要在被装饰的列表中包含下标i，但是包含它会有两个好处：让排序稳定——如果两个项拥有一样的键，那么他们在排序列表中的顺序不会改变。

原始的项不需要是可对比的，因为被装饰的元组的顺序最多被前面两个项决定。举个例子，原始列表可以包含不能直接排序的复数。

对于大型的列表，以及那些计算对比关系的代价很昂贵的列表来说，*在Python的2.4版本之前，DSU几乎是排序这类列表的最快的方法。但是在2.4及之后的版本，key函数就提供了一样的功能了。*